In [14]:
import os

In [15]:
%pwd

'c:\\pawan chinnu data science\\Chest-Cancer-Detection'

In [16]:
os.chdir("../")

In [17]:
%pwd

'c:\\pawan chinnu data science'

In [18]:
os.environ["MLFLOW-TRACKING-URI"]="https://dagshub.com/pawanchinnu17/DeepScan-Automated-Chest-Cancer-Detection-Using-MLflow-and-DVC.mlflow"
os.environ["MLFLOW-TRACKING_USERNAME"]="pawanchinnu17"
os.environ["MLFLOW-TRACKING_PASSWORD"]="afa64adc979875924b96b19bd10958adf02a2ad1"

In [19]:
import tensorflow as tf

In [23]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

OSError: No file or directory found at artifacts/training/model.h5

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/pawanchinnu17/DeepScan-Automated-Chest-Cancer-Detection-Using-MLflow-and-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [ ]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        #self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-03-23 13:01:47,796: INFO: common: yaml file: C:\pawan chinnu data science\Chest-Cancer-Detection\config\config.yaml loaded successfully]
[2024-03-23 13:01:47,799: INFO: common: yaml file: C:\pawan chinnu data science\Chest-Cancer-Detection\params.yaml loaded successfully]
[2024-03-23 13:01:47,801: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 21s 3s/step - loss: 0.0015 - accuracy: 1.0000
[2024-03-23 13:02:09,575: INFO: common: json file saved at: scores.json]


2024/03/23 13:02:10 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-03-23 13:02:12,491: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\gudiv\AppData\Local\Temp\tmpjfw4ztad\model\data\model\assets
[2024-03-23 13:02:13,238: INFO: builder_impl: Assets written to: C:\Users\gudiv\AppData\Local\Temp\tmpjfw4ztad\model\data\model\assets]


c:\pawan chinnu data science\Chest-Cancer-Detection\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
